카카오 api -> 한옥마을 음식점, 한옥마을 카페 등 키워드 검색 -> 위치 가져옴 -> 시각화

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [34]:
ar = pd.read_csv('./address.csv', encoding='utf-8')

In [5]:
ar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   89 non-null     int64  
 1   식당명          89 non-null     object 
 2   도로명주소        89 non-null     object 
 3   지번주소         89 non-null     object 
 4   식당위도         89 non-null     float64
 5   식당경도         89 non-null     float64
 6   Unnamed: 5   0 non-null      float64
 7   Unnamed: 6   0 non-null      float64
 8   Unnamed: 7   0 non-null      float64
 9   Unnamed: 8   0 non-null      float64
 10  Unnamed: 9   0 non-null      float64
 11  Unnamed: 10  0 non-null      float64
 12  Unnamed: 11  0 non-null      float64
 13  Unnamed: 12  0 non-null      float64
 14  Unnamed: 13  0 non-null      float64
 15  Unnamed: 14  0 non-null      float64
 16  Unnamed: 15  0 non-null      float64
 17  Unnamed: 16  0 non-null      float64
 18  Unnamed: 17  0 non-null      float64
 19  Unnamed: 1

In [7]:
import folium as g
import json

In [35]:
g_map = g.Map(location=[35.81563488182217, 127.15352107073629], zoom_start=17)

for i in range(len(ar)):
    marker1=g.Marker([ar.loc[i]['식당위도'], ar.loc[i]['식당경도']],
                    icon = g.Icon(color='red'))
    marker1.add_to(g_map)
g_map

위의 데이터로 부족함 데이터크롤링 필요함

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests; from urllib.parse import urlparse

In [12]:
def search(find):
    time.sleep(10)

    driver = webdriver.Chrome(
        executable_path= "./chromedriver.exe"
    )
    url = "https://m.map.naver.com/"
    driver.get(url)
    action = ActionChains(driver)

    time.sleep(4)
    driver.find_element_by_class_name('Nbox_input_text').click()

    driver.find_element_by_class_name('Nbox_input_text._search_input').send_keys(find)
    driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div[1]/form/div/div[2]/div/span[2]/button[2]').click()

    time.sleep(4)
    replys =driver.find_elements_by_xpath('//*[@id="ct"]/div[2]/ul/li')
    print(len(replys))

    results = []
    for index, reply in enumerate(replys):
            name = reply.find_element_by_css_selector('div.item_tit').text
            address =reply.find_element_by_css_selector('div.wrap_item').text.split('\n')[2]
            results.append((name, address))
    return results

In [13]:
kew = ['전주한옥마을 음식점', '전주한옥마을 카페']
dfvari = ['restaurant','caffe']
for i in range(len(kew)):
    dfvari[i] = search(kew[i])

75
75


In [45]:
restaurant = pd.DataFrame(dfvari[0], columns=['name','address'])
caffe = pd.DataFrame(dfvari[1], columns=['name','address'])

In [29]:
# class Add_to_geo(df)
#     def lattitute(df):
#         global lat
#         addre=df.iloc[:,1]
#         url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' +addre[i]
#         result = requests.get(urlparse(url).geturl(),
#                               headers = {'Authorization': ''})
#         json_obj = result.json()
#         for document in json_obj['documents']:
#             lat = document['y']
#         return lat
    
#     def longtitude(df):
#         global long
#         addre=df.iloc[:,1]
#         url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' +addre[i]
#         result = requests.get(urlparse(url).geturl(),
#                               headers = {'Authorization': ''})
#         json_obj = result.json()
#         for document in json_obj['documents']:
#             long = document['x']
#         return long
    
#     def app_df(lat, long):

In [3]:
res = pd.read_excel('./res.xlsx')
caffe = pd.read_excel('./caffe.xlsx')

In [31]:
res.tail()

,name,address,Latitude,Longitude
70,한국관한옥마을점한식,전라북도 전주시 완산구 태조로 31,35.814294,127.151542
71,한옥가 백련마을한식,전라북도 전주시 완산구 한지길 95,35.816134,127.154186
72,이탈리돈돈가스,전라북도 전주시 완산구 간납로 4,35.816660,127.155153
73,"마닐마닐,마닐마닐 테라스카페","전라북도 전주시 완산구 전동성당길 82 마닐마닐, 마닐마닐테라스",35.815310,127.148800
74,"카페미오카페,디저트",전라북도 전주시 완산구 어진길 104,35.816768,127.148982


In [4]:
res.columns

Index(['name', 'address', 'Latitude', 'Longitude'], dtype='object')

In [47]:
res_1 = pd.concat([res, caffe], ignore_index=True, axis=0)
res_1 = res_1.drop_duplicates(['name'], keep = 'last')
res_1.to_excel('./res_1.xlsx')

In [38]:
g_map = g.Map(location=[35.81563488182217, 127.15352107073629], zoom_start=17)


for i in range(len(res_1)):
    marker1=g.Marker([res_1.loc[i]['Latitude'], res_1.loc[i]['Longitude']],
                    icon = g.Icon(color='red'),
                    popop = res_1.loc[i]['name']).add_to(g_map)    
g_map

In [49]:
res_2 = pd.read_excel('./res_1.xlsx')
res_2.head()

,name,address,Latitude,Longitude,category
0,양반가한정식,전라북도 전주시 완산구 최명희길 30-2 양반가,35.816128,127.151237,음식점
1,다우랑만두,전라북도 전주시 완산구 태조로 33,35.815322,127.149792,음식점
2,한국집비빔밥,전라북도 전주시 완산구 어진길 119,35.816394,127.148275,음식점
3,메르밀진미집 본점국수,전라북도 전주시 완산구 전주천동로 94 메르밀진미집 본점,35.811185,127.150471,음식점
4,45년전통비빔밥종로회관비빔밥,전라북도 전주시 완산구 전동성당길 98,35.814846,127.148775,음식점


In [50]:
g_map = g.Map(location=[35.81563488182217, 127.15352107073629], zoom_start=17)


for i in range(len(res_2)):
    lat = res_2.loc[i, 'Latitude']
    long = res_2.loc[i, 'Longitude']
    color = 'red'
    
    if res_2.loc[i, 'category'] == '카페':
        color = 'blue'
        
    g.CircleMarker([lat,long], color=color, radius = 5).add_to(g_map)

g_map